### 抓取微信公众号文章

- 作者：DGC'Idea
- 版本：V0.1   
- 更新日期：2018年4月14日

----------

- 研究目的：获取微信公众号文章；

In [1]:
#引入包
import pandas as pd
import requests,re,json,time,os

# 自定义
import sys
sys.path.append('/home/jquser')

from dm import *
from tl import exists_file

# 数据目录
_data_path='../../Data/'
# 数据引擎
_data=Csv(_data_path)



#时间戳转换成日期
def timestamp_date(timestamp):
    return time.strftime('%Y-%m-%d',time.localtime(timestamp))


#检测文件      
def exists_weixin_file(user_id,data_root=_data_path):
    #cvs文件名
    data_file='%s%s.csv'%(data_root,'msg_weixin_'+user_id)
    return exists_file(data_file)


#读取数据        
def read_weixin_data(user_id):
    #文件名
    data_file='msg_weixin_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    return df


#保存数据        
def save_weixin_data(user_id,df):
    #文件名
    data_file='msg_weixin_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)
    
    
        

    
#公众号列表    
weixin_list=[
    {
    'user_id':'长赢指数投资',  
    'cookies':'pt2gguin=o0095217585; RK=oNDpmhegNG; ptcz=748df478edcd09e705759a7395c0611daa653bb1e10069c7f06dcfa5e4e2b925; pgv_pvi=2400647168; luin=o0095217585; lskey=0001000013011ace27d808d4cdd1d971a818f9aea90ac12d2ed99025aa2a562fbef3d0bb5a1316d456003732; uin=o0095217585; ptisp=cm; skey=@bNOQXCx2i; ua_id=dKZPqZLYkKSSxMUdAAAAAB0tCEG6h-wy6U3mhalZBIw=; mm_lang=zh_CN; pgv_si=s9808291840; cert=T0QH1N1furGXDz2flotNpWYMBOH_07FQ; uuid=e8611c62a04cf635c8c1d14a861e0224; ticket=097420075816fad4fcf47d1bb490c7f029272a80; ticket_id=gh_de0fe3e80e6f; noticeLoginFlag=1; remember_acct=longrenad; data_bizuin=3014232960; bizuin=3091266278; data_ticket=WMusWzp0Nuoxt7Qd/jokvBtD13s4ceokCAkZijVnVssQzrwFRPXsj9tDtiUsQoZ9; slave_sid=UWN1S2Zxckg2M2J1Q0N5a3dsVXU2TmtwODIxNl9YRXE4N1NmanE3Yk1acHBaVjdlM1drQ2IwVXIyY3Rjd09WOFFMME0zRFBZSnBBN0tjSEh3YmprVVFtczVmRXRqVHlCTmJPUjg4ampZcVVuS1h1TjM3MUZ3dWtoM0dJb3RyVXFUWWJxQlpNT3A5eFp6VHpv; slave_user=gh_de0fe3e80e6f; xid=b1d45ec66976f13f2a3844ffc91fbfb4; openid2ticket_ogIups-z1QLb-nFec80flwcDD8vE=GU73NBUZtgyKTuXEI01iIHIYq4CsJL2SokLeiYKZuk0='
    },
    {
    'user_id':'股市药丸',  
    'cookies':'noticeLoginFlag=1; RK=oNDpmhegNG; ptcz=748df478edcd09e705759a7395c0611daa653bb1e10069c7f06dcfa5e4e2b925; pgv_pvi=2400647168; ua_id=dKZPqZLYkKSSxMUdAAAAAB0tCEG6h-wy6U3mhalZBIw=; mm_lang=zh_CN; pgv_pvid=4337168800; o_cookie=95217585; gr_user_id=6e9dd211-dc9e-48b9-afd2-94bea57f736d; grwng_uid=5d048f38-c639-4c52-b2e5-4caa89adf7cd; 8c66aca9f0d1ff2e_gr_session_id=1e5a4e42-1481-41f8-b182-a589c1e22728; 8c66aca9f0d1ff2e_gr_session_id_1e5a4e42-1481-41f8-b182-a589c1e22728=true; uin=o0095217585; skey=@E1miGYogs; ptisp=cm; rewardsn=; wxtokenkey=777; pgv_si=s2027986944; uuid=ba8b585cf21e28d11ad53da506344cb4; ticket=4f2872e62e1a937a3c29b5a38cbeaf6a1c973367; ticket_id=gh_de0fe3e80e6f; cert=vsfYZ07Y_I4CKiXUtE3X83h1H21Bf2Vb; data_bizuin=3014232960; bizuin=3091266278; data_ticket=UmkbWRQ5LvbfroQwl/F2WeTuCeXu9JGV4lQFwU60ASmdQE0Naj3P4QMaXI593LEK; slave_sid=a3hONko2cnZOVXhFQTViUUtueFo3TmJ2WW1XYmZUc1Q1V2dVaEtBaGlWQ3ZHekFENUVSVElxVWx3RVZJcWRCclFtWFBNanFRVTBiSkJaSzlUUUtHanZjMklwNlJ2VTJzaHA5R3FiRThWOGlrR29ZT0VMYzdDNEl0VldSQ1RHeTE2Z2dacWhSNk1IT0dES3Zl; slave_user=gh_de0fe3e80e6f; xid=5ffc81d72c34207f7fb44ea31c529974; openid2ticket_ogIups-z1QLb-nFec80flwcDD8vE=ICGUDYGrlYa1NBQaddDr9h0mK1RZv6swd45KUz3zCgs='
    },
    {
    'user_id':'阿甘数量化',  
    'cookies':'noticeLoginFlag=1; remember_acct=longrenad; pt2gguin=o0095217585; RK=oNDpmhegNG; ptcz=748df478edcd09e705759a7395c0611daa653bb1e10069c7f06dcfa5e4e2b925; pgv_pvi=2400647168; luin=o0095217585; lskey=0001000013011ace27d808d4cdd1d971a818f9aea90ac12d2ed99025aa2a562fbef3d0bb5a1316d456003732; ua_id=dKZPqZLYkKSSxMUdAAAAAB0tCEG6h-wy6U3mhalZBIw=; mm_lang=zh_CN; noticeLoginFlag=1; remember_acct=longrenad; pgv_pvid=4337168800; o_cookie=95217585; gr_user_id=6e9dd211-dc9e-48b9-afd2-94bea57f736d; grwng_uid=5d048f38-c639-4c52-b2e5-4caa89adf7cd; openid2ticket_ogIups-z1QLb-nFec80flwcDD8vE=GUf5q5pCnvUAQhIh0ZLmYt02Vpqv7DSAk+htlUQHJno=; uin=o0095217585; skey=@xbLftPIhW; ptisp=cm; pgv_si=s4765463552; uuid=e9e85bbfc6366e5b3900d0492e430efb; data_bizuin=3014232960; bizuin=3091266278; data_ticket=uUYW23hv1bPWBM+npgg9rox3yhnaRmfITSDaomVvVQ9yLsSeQebEsp5Fp0+pAh49; slave_sid=d3pYN08ySjNyTXNpcjRQaDJVNklxcFdLWmF1VUFXN1Fvd1NxWElIcVpVYzdBS1g5d3lTaEhqRXNRdWR3YW1sWG9oSkE5VnF3VFRhR3c4OHNTWHF0RUp5ZTBHNGVReWt1azB5QmhzemJRdkFjNDcxUFU3bzZJNWs3V3ZsbHY5bHFnc1F3M0tFSkQyNDZma1JL; slave_user=gh_de0fe3e80e6f; xid=44154e10da22803d6602e5f666d23691'
    },
    {
    'user_id':'梧桐拾贝社',  
    'cookies':'noticeLoginFlag=1; remember_acct=longrenad; pt2gguin=o0095217585; RK=oNDpmhegNG; ptcz=748df478edcd09e705759a7395c0611daa653bb1e10069c7f06dcfa5e4e2b925; pgv_pvi=2400647168; luin=o0095217585; lskey=0001000013011ace27d808d4cdd1d971a818f9aea90ac12d2ed99025aa2a562fbef3d0bb5a1316d456003732; ua_id=dKZPqZLYkKSSxMUdAAAAAB0tCEG6h-wy6U3mhalZBIw=; mm_lang=zh_CN; noticeLoginFlag=1; remember_acct=longrenad; pgv_pvid=4337168800; o_cookie=95217585; uin=o0095217585; skey=@cB3XIsxc5; ptisp=cm; pgv_si=s7381355520; uuid=17d7c627aaf318ce1d9adc84c09f843b; ticket=db05b7ee2eb35ea208dc36d1c69e5abcf16a0603; ticket_id=gh_de0fe3e80e6f; cert=Z6H0_fpJLXzPLKyVlsKJgpZ7ZIv3AIdZ; data_bizuin=3014232960; bizuin=3091266278; data_ticket=MjfV7I5MLoL2aaL398CKvcPdmJheT/ACeNL1cIHqZfNPc5LIHDAx3jzO20HVfbXN; slave_sid=T1FVMGNsYVVlclZiTVdidmhNd3dyakdOR0FFRU9QcFNnUU5hN1haSTdnYWVYckY4U21EWFdheVlSSlZ6WUVxTGdEY3NRUGx2ZDAybDZjNXo2bzhMbldKX3NjQmRyWDhlTTc4ajdOZkw5ZGxfQXE4TmZnWlhxV2FUeUpuQlE1eFhMSDJBVmZrU0dldFNiM3A0; slave_user=gh_de0fe3e80e6f; xid=99455781d7d655d7d6de071fbf529288; openid2ticket_ogIups-z1QLb-nFec80flwcDD8vE=8fMHm2O+R+HP4tN5yQY4UtICv02XdNOZseDWAxIkdIM='    },
    {
    'user_id':'码农滚雪球',  
    'cookies':'noticeLoginFlag=1; remember_acct=longrenad; pt2gguin=o0095217585; RK=oNDpmhegNG; ptcz=748df478edcd09e705759a7395c0611daa653bb1e10069c7f06dcfa5e4e2b925; pgv_pvi=2400647168; luin=o0095217585; lskey=0001000013011ace27d808d4cdd1d971a818f9aea90ac12d2ed99025aa2a562fbef3d0bb5a1316d456003732; ua_id=dKZPqZLYkKSSxMUdAAAAAB0tCEG6h-wy6U3mhalZBIw=; mm_lang=zh_CN; noticeLoginFlag=1; remember_acct=longrenad; pgv_pvid=4337168800; o_cookie=95217585; uin=o0095217585; skey=@cB3XIsxc5; ptisp=cm; pgv_si=s7381355520; uuid=17d7c627aaf318ce1d9adc84c09f843b; ticket=db05b7ee2eb35ea208dc36d1c69e5abcf16a0603; ticket_id=gh_de0fe3e80e6f; cert=Z6H0_fpJLXzPLKyVlsKJgpZ7ZIv3AIdZ; data_bizuin=3014232960; bizuin=3091266278; data_ticket=MjfV7I5MLoL2aaL398CKvcPdmJheT/ACeNL1cIHqZfNPc5LIHDAx3jzO20HVfbXN; slave_sid=T1FVMGNsYVVlclZiTVdidmhNd3dyakdOR0FFRU9QcFNnUU5hN1haSTdnYWVYckY4U21EWFdheVlSSlZ6WUVxTGdEY3NRUGx2ZDAybDZjNXo2bzhMbldKX3NjQmRyWDhlTTc4ajdOZkw5ZGxfQXE4TmZnWlhxV2FUeUpuQlE1eFhMSDJBVmZrU0dldFNiM3A0; slave_user=gh_de0fe3e80e6f; xid=99455781d7d655d7d6de071fbf529288; openid2ticket_ogIups-z1QLb-nFec80flwcDD8vE=8fMHm2O+R+HP4tN5yQY4UtICv02XdNOZseDWAxIkdIM='
    },
]        

# print(' 已准备好！')

数据引擎：运行于研究


In [2]:
#爬取微信公众号文章
def get_weixin_link(query,cookies_str,start=0):
    #公众号主页
    url='https://mp.weixin.qq.com'

    #设置headers
    header={
        "HOST":"mp.weixin.qq.com",
        'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 Safari/537.36 Core/1.53.4620.400 QQBrowser/9.7.13014.400'
        }

    #解析cookies
    cookies_str=cookies_str.split('; ')
    cookies={}
    for item in cookies_str:
        temp=item.split('=')
        cookies[temp[0]]=temp[1]
    
    #获取token
    #登录之后的微信公众号首页url变化为：https://mp.weixin.qq.com/cgi-bin/home?t=home/index&lang=zh_CN&token=1849751598，从这里获取token信息
    response=requests.get(url=url,cookies=cookies)
    token=re.findall(r'token=(\d+)',str(response.url))[0]
     
    #获取fakeid
    #搜索微信公众号的接口地址
    search_url='https://mp.weixin.qq.com/cgi-bin/searchbiz?'
    #搜索微信公众号接口需要传入的参数，有三个变量：微信公众号token、随机数random、搜索的微信公众号名字
    query_id={
        'action':'search_biz',
        'token':token,
        'lang':'zh_CN',
        'f':'json',
        'ajax':'1',
        'random':random.random(),
        'query':query,
        'begin':'0',
        'count':'5'
        }  
    #打开搜索微信公众号接口地址，需要传入相关参数信息如：cookies、params、headers
    search_response=requests.get(search_url,cookies=cookies,headers=header,params=query_id)
    #取搜索结果中的第一个公众号
    lists=search_response.json().get('list')[0]
    #获取这个公众号的fakeid，后面爬取公众号文章需要此字段
    fakeid=lists.get('fakeid')
    
    #搜索文章需要传入几个参数：登录的公众号token、要爬取文章的公众号fakeid、随机数random
    query_id_data={
        'token':token,
        'lang':'zh_CN',
        'f':'json',
        'ajax':'1',
        'random':random.random(),
        'action':'list_ex',
        'begin':'0',#起始位置
        'count':'5',#每次获取的条目数，此参数变化，变化规则为每页加5
        'query':'',#公众号名称
        'fakeid':fakeid,
        'type':'9'
        }
    
    #微信公众号文章接口地址
    appmsg_url='https://mp.weixin.qq.com/cgi-bin/appmsg?'
    #数据列表
    data_list=[]
    df=pd.DataFrame()
    try:
        #获取文章总数
        #打开搜索的微信公众号文章列表页
        appmsg_response=requests.get(appmsg_url,cookies=cookies,headers=header,params=query_id_data)
        appmsg_response=appmsg_response.json()
        if appmsg_response['base_resp']['err_msg']=='freq control':
            print ('获取频率过繁，系统拒绝！')
        #获取文章总数
        max_num=int(appmsg_response.get('app_msg_cnt'))
        #每页至少有5条，获取文章总的页数，爬取时需要分页爬
        pages=int((max_num-start)/5)
        if max_num%5!=0 or pages==0 :
            pages+=1
        #起始页begin参数，往后每页加5
        begin=start
        count=1
        while pages>0:
            #设置每次起始位
            query_id_data['begin']='{}'.format(str(begin))
            #获取每一页文章的标题和链接地址
            query_fakeid_response=requests.get(appmsg_url,cookies=cookies,headers=header,params=query_id_data)
            #获得当前页文章列表
            fakeid_list=query_fakeid_response.json().get('app_msg_list')
            #遍历当前页文章列表
            for item in fakeid_list:
                #文章链接、标题、时间戳（时间戳转换为日期）
                data_list.append([item.get('title'),timestamp_date(item.get('update_time')),item.get('link'),'*',False])
                print(' 已获取：第 %s 篇，共 %s 篇'%(count,max_num),end='\r')
                #计数
                count+=1
            #页面数递减    
            pages-=1
            #起始位置递增
            begin+=5
            #延时5秒，防止被踢
            time.sleep(5)
    except Exception as e:
#         print('错误：%s'%(e))
        pass
    if len(data_list)==0:
        return None
    else:
        #整理数据    
        df=pd.DataFrame(data_list,columns=['title','date','link','content','sended'])    
        #返回数据
        return df

## 1.设置当前公众号

In [3]:
user_id=weixin_list[1]['user_id'] 
cookies=weixin_list[1]['cookies']
print(' 公众号名 ',user_id)

 公众号名  股市药丸


## 2.获取文章链接

In [4]:
#爬取微信公众号文章
def get_weixin_link(query,cookies_str,start=0):
    #公众号主页
    url='https://mp.weixin.qq.com'

    #设置headers
    header={
        "HOST":"mp.weixin.qq.com",
        'User-Agent':'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.104 Safari/537.36 Core/1.53.4620.400 QQBrowser/9.7.13014.400'
        }

    #解析cookies
    cookies_str=cookies_str.split('; ')
    cookies={}
    for item in cookies_str:
        temp=item.split('=')
        cookies[temp[0]]=temp[1]
    
    #获取token
    #登录之后的微信公众号首页url变化为：https://mp.weixin.qq.com/cgi-bin/home?t=home/index&lang=zh_CN&token=1849751598，从这里获取token信息
    response=requests.get(url=url,cookies=cookies)
    token=re.findall(r'token=(\d+)',str(response.url))[0]
     
    #获取fakeid
    #搜索微信公众号的接口地址
    search_url='https://mp.weixin.qq.com/cgi-bin/searchbiz?'
    #搜索微信公众号接口需要传入的参数，有三个变量：微信公众号token、随机数random、搜索的微信公众号名字
    query_id={
        'action':'search_biz',
        'token':token,
        'lang':'zh_CN',
        'f':'json',
        'ajax':'1',
        'random':random.random(),
        'query':query,
        'begin':'0',
        'count':'5'
        }  
    #打开搜索微信公众号接口地址，需要传入相关参数信息如：cookies、params、headers
    search_response=requests.get(search_url,cookies=cookies,headers=header,params=query_id)
    #取搜索结果中的第一个公众号
    lists=search_response.json().get('list')[0]
    #获取这个公众号的fakeid，后面爬取公众号文章需要此字段
    fakeid=lists.get('fakeid')
    
    #搜索文章需要传入几个参数：登录的公众号token、要爬取文章的公众号fakeid、随机数random
    query_id_data={
        'token':token,
        'lang':'zh_CN',
        'f':'json',
        'ajax':'1',
        'random':random.random(),
        'action':'list_ex',
        'begin':'0',#起始位置
        'count':'5',#每次获取的条目数，此参数变化，变化规则为每页加5
        'query':'',#公众号名称
        'fakeid':fakeid,
        'type':'9'
        }
    
    #微信公众号文章接口地址
    appmsg_url='https://mp.weixin.qq.com/cgi-bin/appmsg?'
    #数据列表
    data_list=[]
    df=pd.DataFrame()
    try:
        #获取文章总数
        #打开搜索的微信公众号文章列表页
        appmsg_response=requests.get(appmsg_url,cookies=cookies,headers=header,params=query_id_data)
        appmsg_response=appmsg_response.json()
        
        if appmsg_response['base_resp']['err_msg']=='freq control':
            print ('获取频率过繁，系统拒绝！')
        #获取文章总数
        max_num=int(appmsg_response.get('app_msg_cnt'))
        #每页至少有5条，获取文章总的页数，爬取时需要分页爬
        pages=int((max_num-start)/5)
        if max_num%5!=0 or pages==0 :
            pages+=1
        #起始页begin参数，往后每页加5
        if start==0:
            begin=max_num-4
        else:
            begin=max_num-start-4
        print (begin,pages)    
        count=1
        while pages>0:
            #设置每次起始位
            query_id_data['begin']='{}'.format(str(begin))
            #获取每一页文章的标题和链接地址
            query_fakeid_response=requests.get(appmsg_url,cookies=cookies,headers=header,params=query_id_data)
            
            #获得当前页文章列表
            fakeid_list=query_fakeid_response.json().get('app_msg_list')
            #遍历当前页文章列表
            for item in reversed(fakeid_list):
                #文章链接、标题、时间戳（时间戳转换为日期）
                data_list.append([item.get('title'),timestamp_date(item.get('update_time')),item.get('link'),'*',False])
                print(' 已获取：第 %s 篇，共 %s 篇'%(count,max_num),end='\r')
                #计数
                count+=1
#             print (fakeid_list)
#             1/0    
            #页面数递减    
            pages-=1
            #起始位置递增
            begin-=5
            #延时5秒，防止被踢
            time.sleep(5)
    except Exception as e:
        print('错误：%s'%(e))
        pass
    if len(data_list)==0:
        return None
    else:
        #整理数据    
        df=pd.DataFrame(data_list,columns=['title','date','link','content','sended'])    
        #返回数据
        return df


#断点续爬数据
def update_weixin_data(user_id,cookies):
    #检查是否已存在数据
    if exists_weixin_file(user_id): 
        #已存在数据
        df=read_weixin_data(user_id)
        print(len(df))
        #起始位置为当前记录数
        new_df=get_weixin_link(user_id,cookies,len(df)) 
        if new_df is None:
            print('没有新文章或更新不成功！')
        else:
            #拼接新旧数据
            df=pd.concat([df,new_df],ignore_index=True)
            #保存到cvs文件
            save_weixin_data(user_id,df)
            print(' 共更新：%s 篇文章！'%len(new_df))    
    else:
        #数据不存在，起始位置为0 
        df=get_weixin_link(user_id,cookies,0)  
        if df is None:
            print('获取不成功！')
        else:
            #保存到cvs文件
            save_weixin_data(user_id,df)
            print(' 共获取：%s 篇文章！'%len(df))
    
update_weixin_data(user_id,cookies)

362


IndexError: list index out of range

## 3.获取文章内容

In [10]:
def get_weixin_content(user_id):
    isget=False
    
    if exists_weixin_file(user_id):
        df=read_weixin_data(user_id)  
        for i in range(len(df)):
            if df.content.iloc[i]=='*' :
                print(' 正在获取：%s %s'%(df.date.iloc[i],df.title.iloc[i]),end='\r')
                try:
                    #获取url
                    url=df.link.iloc[i]
                    #请求数据
                    r=requests.get(url)
                    r=r.text
                    #解析网页
                    content=r.split('id="js_content">')[1].split('</div>')[0] 
                    content=content.replace('\r','').replace('\n','').lstrip()
                except:
                    continue
                #添加数据
                df.content.iloc[i]=content
                isget=True
                #延时5秒，防止被踢
                time.sleep(5)
    if isget==True:    
        save_weixin_data(user_id,df) 
        print(' 文章内容获取完毕！')
    else:
        print(' 尚未获取文章列表或是没有要获取的！')
        
df=get_weixin_content(user_id)        
df

/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


 文章内容获取完毕！-12-25 指数基金定投漫谈：估值和概率，应对风险和不确定性对比数据）ct you can prepare)


## 4.发送邮件

In [11]:
import sys

#引入自定义库
sys.path.append("/home/jquser/")
from tl_mail import *

#发送邮件
def send_weixin_data(user_id):
    issend=False
    if not exists_weixin_file(user_id): 
        print(' 未检测到数据文件！')
        return
    #读取数据
    df=read_weixin_data(user_id)
    i=len(df)-1
    while i>=0: 
        if df['sended'].iloc[i]==False:
            #邮件主题
            subjiect='%s（微信文章 %s）'%(df.title.iloc[i],user_id)
            #邮件正文
            message='<h2>%s</h2><p>%s</br>%s</p><hr />%s'%(
                df.title.iloc[i],df.date.iloc[i],df.link.iloc[i],df.content.iloc[i])
            try:
                #发送邮件
                send_html_qqmail(subjiect,message)
                #发送成功后标记
                df['sended'].iloc[i]=True
                issend=True
                #进度
                print(' 已发送：%s'%(subjiect),end='\r')
                #延时5秒，防止被踢
                time.sleep(5)
            except Exception as e:
                print(' 错误：%s'%e)
                break
        i-=1    
    if issend==True:    
        save_weixin_data(user_id,df) 
        print(' 发送完毕！')
    else:
        print(' 没有需要发送的！')
    
send_weixin_data(user_id)     

/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


 发送完毕！享图片（微信文章 梧桐拾贝社）计划（微信文章 梧桐拾贝社）拾贝社）u can prepare)（微信文章 梧桐拾贝社）


## 5.查看数据表

In [5]:
pd.set_option('display.max_rows',2000)
df=read_weixin_data(user_id)
#df['sended'].iloc[35:143]=True
# df['sended']=False
#save_weixin_data(user_id,df)
df

,title,date,link,content,sended
0,全面牛市系列之6：行业大轮动，再无压舱石,2018-12-05,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" data-mpa-powered-b...",True
1,全市场PB破2的第15周，PB重回1.9以下--珍惜好日子（投资周记20181201）,2018-12-01,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" style=""padding-rig...",True
2,2019年A股市场展望：隧道的微光,2018-12-01,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,NaN,True
3,全面牛市系列之5：攻击手们轮动大作战,2018-11-24,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" data-mpa-powered-b...",True
4,全市场PB破2的第14周，PB重回1.9以下--珍惜好日子（投资周记20181124）,2018-11-24,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" style=""padding-rig...",True
5,全面牛市系列之4：寻找行业指数中的攻击手,2018-11-21,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" data-mpa-powered-b...",True
6,想要抄到底该怎么做？,2018-11-17,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" data-mpa-powered-b...",True
7,全市场PB破2的第13周，PB1.9以下都没有了--珍惜好日子（投资周记20181117）,2018-11-17,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" style=""padding-rig...",True
8,全面牛市系列之3：寻找宽基指数中的攻击手,2018-11-11,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" data-mpa-powered-b...",True
9,全市场PB破2的第12周，PB已涨回1.8--珍惜好日子（投资周记20181110）,2018-11-11,http://mp.weixin.qq.com/s?__biz=MzIxODU1OTQ3NA...,"<section class=""article135"" style=""padding-rig...",True
